# Discret logarithmic problème

In [1]:
import sympy as sp
import numpy as np
import math

## Dans le groupe additif $( \mathbb{Z}_p, + )$

étant donné un nombre premier \(p\), un générateur \(g\) du groupe additif $\mathbb{Z}_p$, et un élément $y \in \mathbb{Z}_p$, trouver $x$ tel que :
    $a \cdot x \equiv b \pmod{p}$

In [2]:
# variables globales
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
dict_variable_model_map={}
x=sp.Add()
k=sp.Add()

In [3]:
# Créer un symbole à partir d'un nom en string
def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

In [4]:
# set variable en binaire qui est dans {0,...,p-1}
# exemple : x = u1 + 2u2 + 4u3 + ... + 2^(n-2)un-1 + (p-2^(n-1)+1)un

def set_x_expression(n,p):  # n est length de x en binaire
    # len(x) = len(p)
    expr_x = 0
    # de u1 à un-1
    for i in range(1,n):
        var_name = ('u' + str(i))
        var = create_variables(var_name)
        expr_x = 2**(i-1) * var + expr_x
    # un
    expr_x = expr_x + (p-2**(n-1)+1) * create_variables('u' + str(n))
    return expr_x

def set_k_expression(n):  # l est length de k en binaire
    l = math.floor(math.log2(n)) + 1
    expr_k = 0
    # de u1 à un-1
    for i in range(1,l):
        var_name = ('k' + str(i))
        var = create_variables(var_name)
        expr_k = 2**(i-1) * var + expr_k
    # un
    expr_k = expr_k + (n-2**(l-1)+1) * create_variables('k' + str(n))
    return expr_k

Ici on donne n = 3 comme parapmètre mais le n de x est décidé par p, x et p ont la même taille

In [5]:
x = set_x_expression(3,5)
x

u1 + 2*u2 + 2*u3

### définir fonction
$(x \cdot a \bmod p ) + (-b \bmod p) - k\cdot p = 0 $

In [6]:
def define_function(a, b, p):
    # set variables for x
    n = math.ceil(math.log2(p))  # len de x == len de p
    global x
    x = set_x_expression(n,p)

    #set variables for k
    global k
    k = set_k_expression(n)

    # set expression
    expr = x * sp.Mod(a,p)
    expr = expr + sp.Mod(-b,p)
    expr = expr - (k * p)
    expr = (expr) **2
    expr = sp.expand(expr)
    return expr

In [7]:
expr = define_function(2,3,5)
expr

u1 + 2*u2 + 2*u3
k1 + 2*k3


25*k1**2 + 100*k1*k3 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 - 20*k1 + 100*k3**2 - 40*k3*u1 - 80*k3*u2 - 80*k3*u3 - 40*k3 + 4*u1**2 + 16*u1*u2 + 16*u1*u3 + 8*u1 + 16*u2**2 + 32*u2*u3 + 16*u2 + 16*u3**2 + 16*u3 + 4

In [8]:
def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

In [9]:
expr_reduct_power = reduct_variable_power_function(expr)
expr_reduct_power

100*k1*k3 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 + 5*k1 - 40*k3*u1 - 80*k3*u2 - 80*k3*u3 + 60*k3 + 16*u1*u2 + 16*u1*u3 + 12*u1 + 32*u2*u3 + 32*u2 + 32*u3 + 4

In [10]:
# les varaiables sont sort par la lettre puis un nombre: "x12" -> "x" 12
def sort_key(variable):
    name = str(variable)
    # Split le nom du variable aux deux parties: letter & number 
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):    
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]
    
    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1 
    else:
        x_4 = key 

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :  
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste) 
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_expression(new_expr)
    else:
        return new_expr
    
# obtient l'expression finale
def get_reduct_expression(a,b,p):
    expr = define_function(a,b,p)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
    return reduct_demension_expr

In [11]:
reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
print(reduct_demension_expr)
reduct_demension_expr

100*k1*k3 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 + 5*k1 - 40*k3*u1 - 80*k3*u2 - 80*k3*u3 + 60*k3 + 16*u1*u2 + 16*u1*u3 + 12*u1 + 32*u2*u3 + 32*u2 + 32*u3 + 4


100*k1*k3 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 + 5*k1 - 40*k3*u1 - 80*k3*u2 - 80*k3*u3 + 60*k3 + 16*u1*u2 + 16*u1*u3 + 12*u1 + 32*u2*u3 + 32*u2 + 32*u3 + 4

# transfert to Ising

In [12]:
def transfers_qubo2Ising(reduct_demension_expr):

    subs_relations = {(variable, ( variable + 1 ) /2) for variable in variables}

    # Après la substitution, variables dans l'expression sont des variables d'Ising dans {-1,+1}
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

In [13]:
expr_ising = transfers_qubo2Ising(reduct_demension_expr)
expr_ising

25*k1*k3 - 5*k1*u1 - 10*k1*u2 - 10*k1*u3 + 5*k1/2 - 10*k3*u1 - 20*k3*u2 - 20*k3*u3 + 5*k3 + 4*u1*u2 + 4*u1*u3 - u1 + 8*u2*u3 - 2*u2 - 2*u3 + 81/2

# find solution

In [14]:
#import qubo modèle huristique
import dimod
from dwave.preprocessing.composites import SpinReversalTransformComposite

# Compose the sampler
base_sampler = dimod.ExactSolver()
composed_sampler = SpinReversalTransformComposite(base_sampler)
base_sampler in composed_sampler.children

True

In [15]:
def get_solver_parameter(expr_ising):
    linear = {}
    quadratic ={}
    offset = 0

    for term in expr_ising.args:
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]
        if len(term.free_symbols) ==0:
            offset = coef
        if len(term.free_symbols) ==1:
            linear[variables_liste[0].name] = coef
        elif len(term.free_symbols) ==2:
            quadratic[(variables_liste[0].name,variables_liste[1].name)] = coef

    return (linear,quadratic,offset)

def get_solution(expr_ising):
    (linear,quadratic,offset) = get_solver_parameter(expr_ising)
    response = composed_sampler.sample_ising(linear, quadratic)  #response.data() return (solution, energy, num_occurrences)
    solution = next(response.data())[0]
    return solution

In [16]:
solution = get_solution(expr_ising)
solution

{'k3': np.int8(1),
 'u2': np.int8(1),
 'u3': np.int8(1),
 'k1': np.int8(-1),
 'u1': np.int8(-1)}

In [25]:
def calcule_x(solution,x,k):
    dic_varialbe_value = {}
    for var in variables:
        if(solution[var.name] == np.int8(1)):
            dic_varialbe_value[var] = int(1)
        else:
            dic_varialbe_value[var] = int(0)

    x_value = x.subs(dic_varialbe_value)
    k_value = k.subs(dic_varialbe_value)
    
    return (x_value.evalf(),k_value.evalf())


def factorization(a, b, p):
    global variables, variables_Ising, dictionaire_auxiliary_variable, res,x,k
    variables = []
    variables_Ising = []
    dictionaire_auxiliary_variable = {}
    res = 1
    x= sp.Add()
    k= sp.Add()


    reduct_demension_expr = get_reduct_expression(a,b,p)
    expr_ising = transfers_qubo2Ising(reduct_demension_expr)
    solution = get_solution(expr_ising)
    print(solution)
    (x_val,k_val) = calcule_x(solution,x,k)
    x_int = int(x_val)
    k_int = int(k_val)
    print(f" k: {k_int}")
    print(f"La solution de {a} * x = {b} mod {p} est x = {x_int}.")

In [27]:
a=2
b=3
p=5
factorization(a, b, p)

u1 + 2*u2 + 2*u3
k1 + 2*k3
{'k3': np.int8(1), 'u2': np.int8(1), 'u3': np.int8(1), 'k1': np.int8(-1), 'u1': np.int8(-1)}
 k: 2
La solution de 2 * x = 3 mod 5 est x = 4.


In [28]:
a=5
b=5
p=7
factorization(a, b, p)

u1 + 2*u2 + 4*u3
k1 + 2*k3
{'k3': np.int8(-1), 'u3': np.int8(-1), 'k1': np.int8(1), 'u2': np.int8(-1), 'u1': np.int8(1)}
 k: 1
La solution de 5 * x = 5 mod 7 est x = 1.
